# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

L. Xie  ->  Z.-L. Xie  |  ['L. Xie']
D. Christie  ->  D. Christie  |  ['D. Christie']
L. Xie  ->  Z.-L. Xie  |  ['L. Xie']
H. Beuther  ->  H. Beuther  |  ['H. Beuther']
M. Wells  ->  M. Wells  |  ['M. Wells']
K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']
Arxiv has 53 new papers today
          4 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/4 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2504.12541


extracting tarball to tmp_2504.12541...

 done.


L. Xie  ->  Z.-L. Xie  |  ['L. Xie']
D. Christie  ->  D. Christie  |  ['D. Christie']


...

 done.


/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure figures/spear_vs_V_maskdiffuse_part1_all.pdf 
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:707: LatexWarning: Could not find graphic \includegraphics[height=4.5cm, trim={2cm, 0.0cm, 0, 0}, clip=True]{figures/spear_vs_V_maskdiffuse_part1_all.pdf }
  warnings.warn(LatexWarning(f"Could not find graphic {k}"))


/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure figures/recapV_100referee.png 
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:707: LatexWarning: Could not find graphic \includegraphics[width=9cm, trim={0.5cm 3.4cm 1cm 0.7cm}, clip]{figures/recapV_100referee.png }
  warnings.warn(LatexWarning(f"Could not find graphic {k}"))
/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure figures/recapV_allNORADIO.png 
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:707: LatexWarning: Could not find graphic \includegraphics[width=9cm, trim={0.5cm 0 1cm 0.7cm}, clip]{figures/recapV_allNORADIO.png }

Found 74 bibliographic references in tmp_2504.12963/sample631.bbl.
Retrieving document from  https://arxiv.org/e-print/2504.13020


/tmp/ipykernel_3233/2822249172.py:52: LatexWarning: 2504.13020 did not run properly
not a gzip file
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2504.12541-b31b1b.svg)](https://arxiv.org/abs/2504.12541) | **Evolving Atmospheric Ion Escape from Kepler-1649 b and c: Power-Law Trends in Atmospheric Loss**  |
|| H. Li, et al. -- incl., <mark>L. Xie</mark>, <mark>D. Christie</mark> |
|*Appeared on*| *2025-04-18*|
|*Comments*| *19 pages, 6 figures, 3 tables. Submitted to ApJL*|
|**Abstract**|            Rocky planets orbiting M-dwarf stars are prime targets for characterizing terrestrial atmospheres, yet their long-term evolution under intense stellar winds and high-energy radiation remains poorly understood. The Kepler-1649 system, which hosts two terrestrial exoplanets orbiting an M5V star, presents a valuable opportunity to explore atmospheric evolution in the extreme environments characteristic of M-dwarf stellar systems. In this Letter we show that both planets could have retained atmospheres over gigayear timescales. Using a multi-species magnetohydrodynamic model, we simulate atmospheric ion escape driven by stellar winds and extreme ultraviolet radiation from 0.7 to 4.8 Gyrs. The results show that total ion escape rates follow a power-law decline ($\propto \tau^{-1.6}$ for Kepler-1649 b, $\propto \tau^{-1.5}$ for Kepler-1649 c$\,$), with O$^{+}$ dominating atmospheric loss (76.8%-98.7%). The escape rates at 4.8 Gyrs are two orders of magnitude lower than those during the early epochs ($1.9\times10^{27}$ s$^{-1}$ at 0.7 Gyr vs. $3.0\times10^{25}$ s$^{-1}$ at 4.8 Gyrs for planet b$\,$), while planet b consistently exhibits 1.1-1.9$\times$ higher O$^{+}$ escape rates than planet c due to its closer orbit (0.051 AU vs. 0.088 AU). Despite substantial early atmospheric erosion, both planets may still retain significant atmospheres, suggesting the potential for long-term habitability. These findings offer predictive insight into atmospheric retention in M-dwarf systems and inform future JWST observations aimed at refining habitability assessments.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2504.12963-b31b1b.svg)](https://arxiv.org/abs/2504.12963) | **ALMAGAL IV. Morphological comparison of molecular and thermal dust emission using the histogram of oriented gradients (HOG) method**  |
|| C. Mininni, et al. -- incl., <mark>H. Beuther</mark>, <mark>M. Wells</mark> |
|*Appeared on*| *2025-04-18*|
|*Comments*| *Accepted for publication in A&A; 29 pages, 20 figures*|
|**Abstract**|            The study of molecular line emission is crucial to unveil the kinematics and the physical conditions of gas in star-forming regions. Our aim is to quantify the reliability of using individual molecular transitions to derive physical properties of the bulk of the H2 gas, looking at morphological correlations in their overall integrated molecular line emission with the cold dust. For this study we selected transitions of H2CO, CH$_3$OH, DCN, HC$_3$N, CH$_3$CN, CH$_3$OCHO, SO, and SiO and compared them with the 1.38 mm dust continuum emission at different spatial scales in the ALMAGAL sample, that observed a total of 1013 targets covering all evolutionary stages of the high-mass star-formation process and different conditions of clump fragmentation. We used the method of the histogram of oriented gradients (HOG) implemented in the tool astroHOG to compare the morphology of integrated line emission with maps of the 1.38 mm dust continuum emission. Moreover, we calculated the Spearman's correlation coefficient, and compared it with our astroHOG results. Only H$_2$CO, CH$_3$OH, and SO show emission on spatial scales comparable with the diffuse continuum emission. However, from the HOG method, the median correlation of the emission of each of these species with the continuum is only $\sim$24-29%. In comparison with the dense fragments these molecular species still have low values of correlation. On the other hand DCN, HC$_3$N, CH$_3$CN, and CH$_3$OCHO show a good correlation with the dense dust fragments, above 60%. The worst correlation is seen with SiO, both with the extended continuum emission and with compact sources. From the comparison of the results of the HOG method and the Spearman's correlation coefficient, the HOG method gives much more reliable results than the intensity-based coefficient in estimating the level of similarity of the emission morphology.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2504.12863-b31b1b.svg)](https://arxiv.org/abs/2504.12863) | **The impact of ram pressure on cluster galaxies, insights from GAEA and TNG**  |
|| <mark>L. Xie</mark>, et al. |
|*Appeared on*| *2025-04-18*|
|*Comments*| *11 pages, 8 figures, accepted for publication in A&A*|
|**Abstract**|            Ram pressure stripping (RPS) has a non-negligible impact on the gas content of cluster galaxies. We use the semi-analytic model GAEA and the hydro-simulation TNG to investigate whether cluster galaxies suffer a strong RPS that is sufficient to remove a significant fraction of their gas during the first pericentric passage. We estimate that a ram pressure of $10^{-10.5}$, $10^{-12} $, $10^{-13.5} $ $g cm^{-1} s^{-2}$ can remove at most $90\%$, $50\%$, and $20\%$ of the cold gas reservoir from low-mass galaxies with $9<\log M_{\star}/{\rm M}_{\odot} <9.5$, assuming the gas can be stripped instantaneously. We then use this information to divide the phase space diagram into `strong', `moderate', `weak', and `no' RPS zones. By tracing the orbit of galaxies since $2.5R_{vir}$, we find in both GAEA and TNG that about half of the galaxies in Virgo-like halos ($\log M_h / M_{\odot} \sim 14 $) did not suffer strong RPS during the first pericentric passage. In Coma-like halos ($\log M_h / M_{\odot} \sim 15$), almost all galaxies have suffered strong RPS during the first pericentric passage, which can remove all gas from low-mass galaxies but is insufficient to significantly reduce the gas content of more massive galaxies. In general, results from TNG and GAEA are consistent, with the RPS being only slightly stronger in TNG than in GAEA. Our findings suggest that most cluster galaxies will maintain a notable fraction of their gas and continue forming stars after the first pericentric passage, except for those with low stellar mass ($\log M_{\star}/{\rm M}_{\odot} <9.5$) in very massive halos ($\log M_{h}/{\rm M}_{\odot} > 15$).         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2504.13020-b31b1b.svg)](https://arxiv.org/abs/2504.13020) | **Euclid preparation. Estimating galaxy physical properties using CatBoost chained regressors with attention**  |
|| E. Collaboration, et al. -- incl., <mark>K. Jahnke</mark> |
|*Appeared on*| *2025-04-18*|
|*Comments*| *22 pages, 13 figures, 4 tables. Accepted for publication by Astronomy & Astrophysics*|
|**Abstract**|            Euclid will image ~14000 deg^2 of the extragalactic sky at visible and NIR wavelengths, providing a dataset of unprecedented size and richness that will facilitate a multitude of studies into the evolution of galaxies. In the vast majority of cases the main source of information will come from broad-band images and data products thereof. Therefore, there is a pressing need to identify or develop scalable yet reliable methodologies to estimate the redshift and physical properties of galaxies using broad-band photometry from Euclid, optionally including ground-based optical photometry also. To address this need, we present a novel method to estimate the redshift, stellar mass, star-formation rate, specific star-formation rate, E(B-V), and age of galaxies, using mock Euclid and ground-based photometry. The main novelty of our property-estimation pipeline is its use of the CatBoost implementation of gradient-boosted regression-trees, together with chained regression and an intelligent, automatic optimization of the training data. The pipeline also includes a computationally-efficient method to estimate prediction uncertainties, and, in the absence of ground-truth labels, provides accurate predictions for metrics of model performance up to z~2. We apply our pipeline to several datasets consisting of mock Euclid broad-band photometry and mock ground-based ugriz photometry, to evaluate the performance of our methodology for estimating the redshift and physical properties of galaxies detected in the Euclid Wide Survey. The quality of our photometric redshift and physical property estimates are highly competitive overall, validating our modeling approach. We find that the inclusion of ground-based optical photometry significantly improves the quality of the property estimation, highlighting the importance of combining Euclid data with ancillary ground-based optical data. (Abridged)         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2504.12541/./Kb_kc_escape_rate_20250211.png', 'tmp_2504.12541/./6x8_Kb_20250327_200000_modifycolorbar_colorbarforeveryspecies_changecolorbar_-4t4_deleteH.png', 'tmp_2504.12541/./6x8_Kc_20250327_200000_modifycolorbar_colorbarforeveryspecies_changecolorbar_-4t4_deleteH.png']
copying  tmp_2504.12541/./Kb_kc_escape_rate_20250211.png to _build/html/
copying  tmp_2504.12541/./6x8_Kb_20250327_200000_modifycolorbar_colorbarforeveryspecies_changecolorbar_-4t4_deleteH.png to _build/html/
copying  tmp_2504.12541/./6x8_Kc_20250327_200000_modifycolorbar_colorbarforeveryspecies_changecolorbar_-4t4_deleteH.png to _build/html/
exported in  _build/html/2504.12541.md
    + _build/html/tmp_2504.12541/./Kb_kc_escape_rate_20250211.png
    + _build/html/tmp_2504.12541/./6x8_Kb_20250327_200000_modifycolorbar_colorbarforeveryspecies_changecolorbar_-4t4_deleteH.png
    + _build/html/tmp_2504.12541/./6x8_Kc_20250327_200000_modifycolorbar_colorbarforeveryspecies_changecolorbar_-4t4_deleteH.pn

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$</div>



<div id="title">

# Evolving Atmospheric Ion Escape from Kepler-1649 b and c: Power-Law Trends in Atmospheric Loss

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2504.12541-b31b1b.svg)](https://arxiv.org/abs/2504.12541)<mark>Appeared on: 2025-04-18</mark> -  _19 pages, 6 figures, 3 tables. Submitted to ApJL_

</div>
<div id="authors">

H. Li, et al. -- incl., <mark>L. Xie</mark>, <mark>D. Christie</mark>

</div>
<div id="abstract">

**Abstract:** Rocky planets orbiting M-dwarf stars are prime targets for characterizing terrestrial atmospheres, yet their long-term evolution under intense stellar winds and high-energy radiation remains poorly understood. The Kepler-1649 system, which hosts two terrestrial exoplanets orbiting an M5V star, presents a valuable opportunity to explore atmospheric evolution in the extreme environments characteristic of M-dwarf stellar systems. In this Letter we show that both planets could have retained atmospheres over gigayear timescales. Using a multi-species magnetohydrodynamic model, we simulate atmospheric ion escape driven by stellar winds and extreme ultraviolet radiation from 0.7 to 4.8 Gyrs. The results show that total ion escape rates follow a power-law decline ( $\propto \tau^{-1.6}$ for Kepler-1649 b, $\propto \tau^{-1.5}$ for Kepler-1649 c $ $ ), with $\ce{O+}$ dominating atmospheric loss (76.8 \% –98.7 \% ). The escape rates at 4.8 Gyrs are two orders of magnitude lower than those during the early epochs ( $1.9\times10^{27}$ s $^{-1}$ at 0.7 Gyr vs. $3.0\times10^{25}$ s $^{-1}$ at 4.8 Gyrs for planet b $ $ ), while planet b consistently exhibits 1.1–1.9 $\times$ higher $\ce{O+}$ escape rates than planet c due to its closer orbit (0.051 AU vs. 0.088 AU). Despite substantial early atmospheric erosion, both planets may still retain significant atmospheres, suggesting the potential for long-term habitability. These findings offer predictive insight into atmospheric retention in M-dwarf systems and inform future JWST observations aimed at refining habitability assessments.

</div>

<div id="div_fig1">

<img src="tmp_2504.12541/./Kb_kc_escape_rate_20250211.png" alt="Fig1" width="100%"/>

**Figure 1. -** Evolution of ion escape rates for Kepler-1649 b (left panels) and c (right panels) across stellar ages (0.7--4.8 Gyrs).
Top to bottom: \ce{O+}, \ce{O2+}, \ce{CO2+}, and total ion escape rates (s$^{-1}$). Data points are fitted with power-law functions (solid lines).
	 (*fig1*)

</div>
<div id="div_fig2">

<img src="tmp_2504.12541/./6x8_Kb_20250327_200000_modifycolorbar_colorbarforeveryspecies_changecolorbar_-4t4_deleteH.png" alt="Fig2" width="100%"/>

**Figure 2. -** Logarithmic ion number density (cm$^{-3}$) distributions in the X-Z plane for Kepler-1649 b at different stellar ages (0.7--4.8 Gyrs; top to bottom). Columns show \ce{O+}, \ce{O2+}, and \ce{CO2+}(left to right). Coordinates are normalized to planetary radius $R_{\rm P}$.
         (*Fig:2D_distribution_Kb*)

</div>
<div id="div_fig3">

<img src="tmp_2504.12541/./6x8_Kc_20250327_200000_modifycolorbar_colorbarforeveryspecies_changecolorbar_-4t4_deleteH.png" alt="Fig3" width="100%"/>

**Figure 3. -** Logarithmic ion number density (cm$^{-3}$) distributions in the X-Z plane for Kepler-1649 c at different stellar ages (0.7--4.8 Gyrs; top to bottom). Columns show \ce{O+}, \ce{O2+}, and \ce{CO2+}(left to right). Coordinates are normalized to planetary radius $R_{\rm P}$.
         (*Fig:2D_distribution_Kc*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2504.12541"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$</div>



<div id="title">

# ALMAGAL IV. Morphological comparison of molecular and thermal dust emission using the histogram of oriented gradients (HOG) method

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2504.12963-b31b1b.svg)](https://arxiv.org/abs/2504.12963)<mark>Appeared on: 2025-04-18</mark> -  _Accepted for publication in A&A; 29 pages, 20 figures_

</div>
<div id="authors">

C. Mininni, et al. -- incl., <mark>H. Beuther</mark>, <mark>M. Wells</mark>

</div>
<div id="abstract">

**Abstract:** The study of molecular line emission is crucial to unveil the kinematics and the physical conditions of gas in star-forming regions. We utilize data from the ALMAGAL survey which provides an unprecedentedly large statistical sample of high-mass star-forming clumps that helps to remove bias and reduce noise (e.g. due to source peculiarities, selection or environmental effects) to determine how well individual molecular species trace continuum emission. Our aim is to quantify the reliability of using individual molecular transitions to derive physical properties of the bulk of the $H_2$ gas, looking at morphological correlations in their overall integrated molecular line emission with the cold dust. For this study we selected transitions  of $H_2$ CO, $CH_3$ OH, DCN, $HC_3$ N, $CH_3$ CN, $CH_3$ OCHO, SO, and SiO  and compared them with the 1.38 mm dust continuum emission at different spatial scales in the ALMAGAL sample. We included two transitions of $H_2$ CO to understand the validity of the results depending on the excitation condition of the selected transition of a molecular specie. The ALMAGAL project observed more than 1000 candidate high-mass star-forming clumps in ALMA Band 6 at a spatial resolution down to 1000 au. A total of 1013 targets have been analyzed in the present paper, covering all evolutionary stages of the high-mass star-formation process and different conditions of clump fragmentation. We used, for the first time on a large statistical sample, the method of the histogram of oriented gradients (HOG) implemented in the tool \texttt{astroHOG} to compare the morphology of integrated line emission with maps of the 1.38 mm dust continuum emission. For each clump, we defined two masks: one that covers the extended more diffuse continuum emission and a smaller one that contains only the compact sources. We select these two masks to study if and how the correlation among the selected molecules changes with the spatial scale of the emission, from extended more diffuse gas in the clumps to denser gas in compact fragments (cores). Moreover, we calculated the Spearman's correlation coefficient, and compared it with our astroHOG results. Among the molecular species analyzed in this paper, only $H_2$ CO, $CH_3$ OH, and SO show emission on spatial scales comparable with the diffuse 1.38 mm dust continuum emission. However, from the HOG method, the median correlation of the emission of each of these species with the continuum is only $\sim$ 24-29 \% . In comparison with the dusty dense fragments these molecular species still have low values of correlation, on average below 45 \% . The low level of morphological correlation suggests that these molecular lines likely trace on average the clump medium or outer layers around dense fragments (in some cases possibly due to optical depth effects) or are also at this scale tracing inner parts of outflows. On the other hand DCN, $HC_3$ N, $CH_3$ CN, and $CH_3$ OCHO show a good correlation with the dense dust fragments, above 60 \% . The worst correlation is seen with SiO, both with the extended continuum emission and with compact sources. Moreover, unlike other outflow tracers, SiO in a large fraction of the sources does not cover well the area of the extended continuum emission. This and the results of the astroHOG analysis reveal that SiO and SO do not trace the same gas, contrary to what was previously thought. From the comparison of the results of the HOG method and the Spearman's correlation coefficient, the HOG method gives much more reliable results than the intensity-based coefficient in estimating the level of similarity of the emission morphology.

</div>

<div id="div_fig1">

<img src="tmp_2504.12963/./figures/astroHOG_histogram_diffuse_all.png" alt="Fig12.1" width="50%"/><img src="tmp_2504.12963/./figures/astroHOG_histogram_cores_all.png" alt="Fig12.2" width="50%"/>

**Figure 12. -** 
    Morphological correlation between the continuum emission and the line emission (moment-0) for the indicated transition, as quantified by $V_{\rm{N}}$ parameter from the \texttt{astroHOG} method.
    Values of $V_{\rm{N}}$$\approx$ 0 and $V_{\rm{N}}$$\approx$ 1  correspond to no correlation and perfect correlation in the distribution of the two emissions, respectively.
    The upper panels correspond to the correlation on the intersection of the moment-0 line emission mask and the mask$_{\rm{ext.}}$.
    The lower panels correspond to a mask selecting only the compact sources where the line is detected, starting from mask$_{\mathrm{com.}}$.
    The dotted and dashed histograms represent how the distribution would change if all the values were replaced by V$_{\mathrm{N}}- $error (the error on the value of $V_{\mathrm{N}}$ is derived using an MC method, see Sect. 4.3) and $V_{\mathrm{N}}+ $error, respectively.  (*fig:resultsastroHOG*)

</div>
<div id="div_fig2">

<img src="" alt="Fig13.1" width="50%"/><img src="tmp_2504.12963/./figures/spear_vs_V_maskdiffuse_part2_all.png" alt="Fig13.2" width="50%"/>

**Figure 13. -** Comparison between Spearman's correlation coefficient and the $V_{N}$ parameter from the HOG method in the comparison between the continuum and the indicated species using mask$_{\rm{ext.}}$. The two boxes delimited by red dotted-dashed lines are the regions in the plot where the two estimators give the more contrasting results.  (*fig:spearman1*)

</div>
<div id="div_fig3">

<img src="tmp_2504.12963/./figures/grid_properties_detection.png" alt="Fig2.1" width="50%"/><img src="tmp_2504.12963/./figures/grid_properties_detection.png" alt="Fig2.2" width="50%"/>

**Figure 2. -** Increase of the detection statistic of the molecular transitions analyzed, divided in bins of temperature of the clump (upper panel) and evolutionary stage of the clump L/M (lower panel). (*fig:grid_prop*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2504.12963"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

446  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

12  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

3  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
